In [ ]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import matplotlib.pyplot as plt

import torchlensmaker as tlm



class PlacedShape: # call it Surface?
    """
    Wraps a shape with a 'origin' position
    to place it in absolute 2D space.
    """

    def __init__(self, shape, origin):
        self.shape = shape
        self.origin = torch.as_tensor(origin)

    def domain(self):
        return self.shape.domain()

    def evaluate(self, ts):
        "Convert the inner shape evaluate() to absolute space"
        relative_points = self.shape.evaluate(ts)
        return relative_points + self.origin

    def normal(self, ts):
        return self.shape.normal(ts)

    def collide(self, absolute_lines):
        "Collide with lines expressed in absolute space"

        # Convert lines to relative space
        relative_lines = []
        a, b, c = absolute_lines[:, 0], absolute_lines[:, 1], absolute_lines[:, 2]
        new_c = c + a*self.origin[0] + b*self.origin[1]
        relative_lines = torch.column_stack((a, b, new_c))

        # Collide
        relative_points, normals = self.shape.collide(relative_lines)

        # Convert relative points back to absolute space
        return relative_points + self.origin, normals



        
# signal what I want to optimize or not by just wrapping into parameter:

# ReflectiveSurface(Parabola(tlm.Parameter(5.0)))
# vs
# ReflectiveSurface(Parabola(5.0), pos=(a, b))

# setup a linear stack with same syntax gap(), etc.
# but preprocess the stack to remove gaps and setup surfaces origins


# Wrap shapes in PlacedShape()
#     - adds a position(), get it from the accumulated stack data
#     - converts to and from rays coordinates that are relative to the shape

# Render: directly call PlacedShape.evaluate(), PlaceShape.domain(), etc.
#         rays: already in absolute space, render directly


In [ ]:
test_shapes = [
    tlm.BezierSpline(15.0, init=([3.0], [0.2*15., 1.2*15.], [6.8])),
    tlm.Line(15.0, [0.0, 1.0, 0.0]),
    #tlm.PiecewiseLine(18.0, init=[(25/4, 25/2, 3/4*25, 25), (-1.0, -3.0, -3.5, -7.0)]),
    tlm.CircularArc(15.0, init=[-25.]),
    tlm.CircularArc(15.0, init=[25.]),
    tlm.Parabola(10.0, init=[0.002]),
]

test_lines = torch.tensor([
    [1., 0.5, 15.],
    [-9.0, -1.5, -25.],
    [3.6, -5, -20],
    [3., 0., -18.],
])

positions = [
    (0.0, 0.0),
    (15.0, -30.0),
    (-15.0, 35.0),
    (0.0, 15.0),
    (0.0, -15.0),
    (12.51, 5.72),
    (-14.3, -5.1),
]

xlim, ylim = [
    (-50, 50),
    (-50, 50)
]

def render_shape(ax, shape: PlacedShape):
    # Render the shape profile
    points = shape.evaluate(torch.linspace(*shape.domain(), 50)).detach().numpy()
    ax.plot(points[:, 0], points[:, 1], color="steelblue")

    # Add some normals
    normalst = torch.linspace(*shape.domain(), 5)
    normals_origins = shape.evaluate(normalst).detach().numpy()
    normals_vectors = shape.normal(normalst).detach().numpy()
    for o, n in zip(normals_origins, normals_vectors):
        ax.plot([o[0], o[0]+n[0]], [o[1], o[1]+n[1]], color="grey")

    ax.set_aspect("equal")


def render_lines(ax, lines):
    ymin, ymax = ax.get_ylim()
    over = 0.35
    ymin = ymin - over*(ymax - ymin)
    ymax = ymax + over*(ymax - ymin)
    for line in lines:
        a, b, c = line
        x0 = (-c-b*ymin)/a
        x1 = (-c-b*ymax)/a
        ax.plot([x0, x1], [ymin, ymax], color="orange")


def render_points(ax, points):
    if isinstance(points, torch.Tensor):
        points = points.detach().numpy()
    ax.scatter(points[:, 0], points[:, 1], color="green", marker="o")


def test_shape(ax, shape):
    render_shape(ax, shape)

    render_lines(ax, test_lines)

    collisions, normals = shape.collide(test_lines)
    render_points(ax, collisions)

def main():
    for shape in test_shapes:
        fig, ax = plt.subplots(1, 1, figsize=(15, 8))
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

        for origin in positions:
            placed_shape = PlacedShape(shape, origin)
            test_shape(ax, placed_shape)
    
main()